<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/Game_of_thrones_community_iteration/Game%20of%20thrones%20community%20iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Updated to GDS 2.0 version
* Link to original blog post: https://towardsdatascience.com/community-detection-through-time-using-seed-property-in-neo4j-on-the-game-of-thrones-dataset-a2e520a6c79f

In [1]:
!pip install neo4j

     |████████████████████████████████| 89 kB 3.5 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.2-py3-none-any.whl size=115365 sha256=4b6b3195024550cad7621ab4012c1b3dc9630377b161e72e6b622ee512cff2c6
  Stored in directory: /root/.cache/pip/wheels/10/d6/28/95029d7f69690dbc3b93e4933197357987de34fbd44b50a0e4
Successfully built neo4j


In [2]:
from neo4j import GraphDatabase
host = 'bolt://3.235.2.228:7687'
user = 'neo4j'
password = 'seats-drunks-carbon'
driver = GraphDatabase.driver(host,auth=(user, password))

In [30]:
import pandas as pd

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [4]:
from IPython.display import IFrame, HTML
import json
import uuid


def generate_vis(host, user, password, cypher, labels_json, relationships_json):
    html = """\
<html>
<head>
    <title>Neovis.js Simple Example</title>
            <style type="text/css">
                html, body {{
                    font: 16pt arial;
                }}
                #viz {{
                    width: 600px;
                    height: 800px;
                    font: 22pt arial;
                }}
            </style>
            <script src="https://cdn.neo4jlabs.com/neovis.js/v1.1.0/neovis.js"></script>
            <script
                    src="https://code.jquery.com/jquery-3.2.1.min.js"
                    integrity="sha256-hwg4gsxgFZhOsEEamdOYGBf13FyQuiTwlAQgxVSNgt4="
                    crossorigin="anonymous"></script>
            <script type="text/javascript">
                var viz;
                function draw() {{
                    var config = {{
                        container_id: "viz",
                        server_url: "{host}",
                        server_user: "{user}",
                        server_password: "{password}",
                        labels: {labels},
                        relationships: {relationships},
                        initial_cypher: "{cypher}"
                    }};
                    viz = new NeoVis.default(config);
                    viz.render();                    
                    viz.onVisualizationRendered(function(ctx) {{
                        let imageSrc = document.getElementsByTagName("canvas")[0].toDataURL();
                        console.log(imageSrc);
                        document.getElementById("viz-image").src=imageSrc;
                        //document.getElementById("viz").style="display:none";
                        
                        let kernel = IPython.notebook.kernel;
                        //let command = 'display(HTML('<img id="viz-image" width="300px" src="' + imageSrc + '" />';
                        let command = "foo = 'bar'";
                        kernel.execute(command);
                        
                    }});
                }}
            </script>
         </head>
        <body onload="draw()">
            <div id="viz"></div>
        </body>
    </html>
    """

    html = html.format(
        host=host,
        user=user,
        password=password,
        cypher=cypher,
        labels = json.dumps(labels_json),
        relationships=json.dumps(relationships_json)
    )

    unique_id = str(uuid.uuid4())
    filename = "graph-{}.html".format(unique_id)

    with open(filename, "w") as f:
        f.write(html)
    return IFrame(src=filename, width=1000, height=800)


In [5]:
def visualize_level(level, community):
    # Define cypher query
    if level > 1:
        cypher = """MATCH (p1:Person)-[r:INTERACTS_{rel_level}|:INTERACTS_{prev_level}]-(p2:Person) \
                    WHERE p1.community_{level} = {community} RETURN *""".format(
            rel_level=level if level != 4 else 45,level=level, prev_level=level -1, community=community)
    else:
        cypher = """MATCH (p1:Person)-[r:INTERACTS_{level}]-(p2:Person) \
                    WHERE p1.community_{level} = {community} RETURN *""".format(level=level, community=community)
    print(cypher)
    # Define relationships_json
    relationships_json = dict()
    for l in [level-1,level]:
        relationships_json["INTERACTS_{}".format(l if l != 4 else 45)] = {
                "caption": False
            }
    # Define labels_json    
    labels_json = {
        "Person": {
            "caption": "id",
            "community": "community_{}".format(level)
        }
    }

    return generate_vis(host, user, password, cypher, labels_json, relationships_json)

# Import

In [31]:
constraint_query = """CREATE CONSTRAINT IF NOT EXISTS ON (p:Person) ASSERT p.id IS UNIQUE;"""
run_query(constraint_query)

""


In [7]:
# https://networkofthrones.wordpress.com/
import_networks = """

UNWIND ['1','2','3','45'] as book
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book' + book + '-edges.csv' as value
MERGE (source:Person{id:value.Source})
MERGE (target:Person{id:value.Target})
WITH source,target,value.weight as weight,book
CALL apoc.merge.relationship(source,'INTERACTS_' + book, {}, {weight:toFloat(weight)}, target) YIELD rel
RETURN distinct 'done'

"""
run_query(import_networks)

In [32]:
def write_louvain(book):
  project_graph_query = f"""
   CALL gds.graph.project.cypher('book',
  'MATCH (p:Person)
  WHERE (p)-[:INTERACTS_{book}]-()
  RETURN id(p) as id',
  'MATCH (p:Person)-[:INTERACTS_{book}]-(p1:Person)
  RETURN id(p) as source, id(p1) as target')
"""

  louvain_book = f"""
  CALL gds.louvain.write('book'
  ,{{writeProperty:'community_{book}'}})
  """

  drop_graph = """
  CALL gds.graph.drop('book')
  """
  run_query(project_graph_query)
  run_query(louvain_book)
  run_query(drop_graph)

# Book 1

In [33]:
write_louvain("1")

In [36]:
# Get Daenerys' community id 
get_daenerys_community_query = """
MATCH (p:Person{id:'Daenerys-Targaryen'})
RETURN p.community_1 as community
"""

daenerys_community = run_query(get_daenerys_community_query)['community'][0]

In [38]:
visualize_level(level=1,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_1]-(p2:Person)                     WHERE p1.community_1 = 52 RETURN *


# Book 2

In [39]:
write_louvain("2")

In [ ]:
visualize_level(level=2,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_2|:INTERACTS_1]-(p2:Person)                     WHERE p1.community_2 = 3 RETURN *


# Book 3

In [41]:
write_louvain("3")

In [ ]:
visualize_level(level=3,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_3|:INTERACTS_2]-(p2:Person)                     WHERE p1.community_3 = 3 RETURN *


# Book 4

In [43]:
write_louvain("45")

In [ ]:
visualize_level(level=4,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_45|:INTERACTS_3]-(p2:Person)                     WHERE p1.community_4 = 3 RETURN *
